In [1]:
import pandas as pd
import random
import time
from Biscuit import *
from Dough import *
from GeneticAlgorithm import *
from GeneticElitism import * 
from GeneticTournament import *
from UniformCrossoverGA import *

In [2]:
df = pd.read_csv('defects.csv', sep =',')

In [3]:
df.head()

,x,class
0,355.449335,c
1,92.496236,a
2,141.876795,c
3,431.833902,c
4,435.028461,c


In [4]:
biscuit0 = Biscuit(0)
biscuit1 = Biscuit(1)
biscuit2 = Biscuit(2)
biscuit3 = Biscuit(3)
biscuit4 = Biscuit(4)

print(biscuit0)
print(biscuit1)
print(biscuit2)
print(biscuit3)
print(biscuit4)

Biscuit Type: 0, Length: 4, Value: 3, Max Defects: {'a': 4, 'b': 2, 'c': 3}
Biscuit Type: 1, Length: 8, Value: 12, Max Defects: {'a': 5, 'b': 4, 'c': 4}
Biscuit Type: 2, Length: 2, Value: 1, Max Defects: {'a': 1, 'b': 2, 'c': 1}
Biscuit Type: 3, Length: 5, Value: 8, Max Defects: {'a': 2, 'b': 3, 'c': 2}
Biscuit Type: 4, Length: 1, Value: -1, Max Defects: {'a': 10, 'b': 10, 'c': 10}


In [5]:
dough = Dough(length=500)

for _, row in df.iterrows():
    dough.add_defect(position=row['x'], defect_class=row['class'])

In [6]:
print(dough)

Dough Defects:
Position: 355.45, Class: c
Position: 92.50, Class: a
Position: 141.88, Class: c
Position: 431.83, Class: c
Position: 435.03, Class: c
Position: 205.80, Class: a
Position: 34.69, Class: b
Position: 443.57, Class: a
Position: 69.42, Class: a
Position: 301.28, Class: a
Position: 342.65, Class: c
Position: 216.12, Class: a
Position: 367.67, Class: c
Position: 93.13, Class: a
Position: 401.42, Class: c
Position: 74.25, Class: c
Position: 356.84, Class: a
Position: 457.42, Class: a
Position: 395.64, Class: c
Position: 368.89, Class: a
Position: 163.92, Class: a
Position: 484.16, Class: b
Position: 55.27, Class: c
Position: 171.56, Class: b
Position: 119.69, Class: a
Position: 332.64, Class: b
Position: 85.10, Class: a
Position: 289.68, Class: b
Position: 156.79, Class: b
Position: 143.53, Class: c
Position: 83.61, Class: a
Position: 108.52, Class: a
Position: 57.58, Class: c
Position: 9.96, Class: c
Position: 440.00, Class: b
Position: 146.01, Class: b
Position: 259.33, Class:

# Finding the optimum for comparison

In [7]:
# Initialize an empty list for each position in the dough
dough_defects = [[] for _ in range(dough.LENGTH)]  # Represents defects at each position of the dough

# Iterate over each row in the DataFrame to record defects
for _, row in df.iterrows():
    position = int(row['x'])  # Convert the defect position to an integer
    defect_class = row['class']  # Extract the defect class
    dough_defects[position].append(defect_class)  # Append the defect class to the list for that position

# Create a list of Biscuit objects for all 4 types
biscuit_types = [Biscuit(i) for i in range(4)]

# Initialize an empty list to keep track of the best path at each position
maxAtindex = [[] for _ in range(500)]

In [8]:
# Useful functions:
def get_val(roll):
    """
    Get the total value from a given arrangement of biscuits.

    Parameters:
    - roll (list): List of tuples containing (Biscuit, position).

    Returns:
    - int: Sum of the values of all biscuits in the arrangement.
    """
    total_value = 0
    for biscuit, position in roll:
        total_value += biscuit.value
    return total_value

def can_be_placed(biscuit, i, dough_defects=dough_defects):
    """
    Check if a biscuit can be placed at a given index on the dough.

    Parameters:
    - biscuit (Biscuit): The Biscuit object to be placed.
    - i (int): The starting index on the dough.
    - dough_defects (list): List representing defects at each position on the dough.

    Returns:
    - bool: True if the biscuit can be placed, False otherwise.
    """
    count_a = 0
    count_b = 0
    count_c = 0
    for index in range(i, biscuit.length + i):
        count_a += dough_defects[index].count('a')
        count_b += dough_defects[index].count('b')
        count_c += dough_defects[index].count('c')
    
    if (count_a <= biscuit.max_defects['a'] and
        count_b <= biscuit.max_defects['b'] and
        count_c <= biscuit.max_defects['c']):
        return True
    else:
        return False

In [9]:
start = time.time()
for i in range(dough.LENGTH):
    max_val = float("-inf")
    max_roll = []
    for biscuit in biscuit_types:  # Iterate over all biscuit types
        if i >= biscuit.length:
            roll = maxAtindex[i - biscuit.length]  # Current best roll at position i - length of biscuit
            current_val = get_val(roll)

            if can_be_placed(biscuit, i - biscuit.length):  # Check if we can place it
                current_val += biscuit.value

            if current_val >= max_val:  # If we got a better current value, replace it
                max_roll = roll + [(biscuit, i - biscuit.length)]
                max_val = current_val

    maxAtindex[i] = max_roll

print("Found in:", time.time() - start, "seconds")
print("Value of Maximum:", get_val(maxAtindex[-1]))
for biscuit, position in maxAtindex[-1]:
    print(biscuit, "at position:", position)

Found in: 0.009991645812988281 seconds
Value of Maximum: 729
Biscuit Type: 1, Length: 8, Value: 12, Max Defects: {'a': 5, 'b': 4, 'c': 4} at position: 0
Biscuit Type: 1, Length: 8, Value: 12, Max Defects: {'a': 5, 'b': 4, 'c': 4} at position: 8
Biscuit Type: 1, Length: 8, Value: 12, Max Defects: {'a': 5, 'b': 4, 'c': 4} at position: 16
Biscuit Type: 3, Length: 5, Value: 8, Max Defects: {'a': 2, 'b': 3, 'c': 2} at position: 24
Biscuit Type: 3, Length: 5, Value: 8, Max Defects: {'a': 2, 'b': 3, 'c': 2} at position: 29
Biscuit Type: 3, Length: 5, Value: 8, Max Defects: {'a': 2, 'b': 3, 'c': 2} at position: 34
Biscuit Type: 2, Length: 2, Value: 1, Max Defects: {'a': 1, 'b': 2, 'c': 1} at position: 39
Biscuit Type: 2, Length: 2, Value: 1, Max Defects: {'a': 1, 'b': 2, 'c': 1} at position: 41
Biscuit Type: 1, Length: 8, Value: 12, Max Defects: {'a': 5, 'b': 4, 'c': 4} at position: 43
Biscuit Type: 1, Length: 8, Value: 12, Max Defects: {'a': 5, 'b': 4, 'c': 4} at position: 51
Biscuit Type: 1,

# Finding Heuristics

In [10]:
biscuits = [biscuit0,biscuit1,biscuit2,biscuit3]
for biscuit in biscuits:
    print("Biscuit type: ",biscuit.biscuit_type)
    print("Value/Length: ",biscuit.value/biscuit.length)
    maxd = sum(biscuit.max_defects.values())
    print("Max defects: ", maxd)
    print("Value/(Length*Max_Defects): ", biscuit.value/(biscuit.length*maxd), "\n")


Biscuit type:  0
Value/Length:  0.75
Max defects:  9
Value/(Length*Max_Defects):  0.08333333333333333 

Biscuit type:  1
Value/Length:  1.5
Max defects:  13
Value/(Length*Max_Defects):  0.11538461538461539 

Biscuit type:  2
Value/Length:  0.5
Max defects:  4
Value/(Length*Max_Defects):  0.125 

Biscuit type:  3
Value/Length:  1.6
Max defects:  7
Value/(Length*Max_Defects):  0.22857142857142856 



The heuristic tells us the order:

- Biscuit 3 > Biscuit 0 > Biscuit 1 > Biscuit 2

# 1. Genetic Algorithm

In [11]:
#Simple program to launch the Genetic Algorithm
def main():
    dough = Dough(length=500)
    for _, row in df.iterrows():
        dough.add_defect(position=row['x'], defect_class=row['class'])
        
    biscuits = {
        0: Biscuit(0),
        1: Biscuit(1),
        2: Biscuit(2),
        3: Biscuit(3)
    }
    
    GA = GeneticAlgorithm(dough, biscuits, population_size=50, mutation_rate=0.01, crossover_rate=0.7)
    GA.initialize_population()

    for generation in range(100): 
        print(generation)
        GA.evolve()

    best_solution = max(GA.population, key=GA.fitness)
    total_value = GA.fitness(best_solution)
    print("Total Value:", total_value)
    print("Best Solution:", best_solution)
    
    return best_solution


In [12]:
if __name__ == "__main__":
    best_solution = main()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Total Value: 554
Best Solution: [(0, 1), (8, 1), (16, 3), (21, 1), (29, 3), (34, 2), (36, 2), (40, 1), (48, 2), (50, 0), (54, 0), (58, 3), (63, 1), (71, 0), (75, 1), (83, 1), (91, 0), (95, 2), (98, 0), (102, 0), (106, 0), (111, 0), (118, 0), (122, 2), (124, 0), (130, 1), (138, 0), (142, 0), (146, 0), (150, 3), (155, 2), (158, 3), (164, 2), (167, 2), (169, 2), (172, 3), (177, 0), (181, 2), (185, 2), (187, 1), (195, 1), (203, 2), (205, 3), (210, 0), (214, 0), (218, 3), (223, 2), (225, 3), (230, 2), (232, 3), (237, 2), (239, 2), (242, 0), (246, 0), (250, 1), (258, 3), (263, 1), (271, 1), (279, 0), (283, 1), (291, 2), (293, 1), (301, 1), (309, 3), (314, 0), (318, 1), (326, 0), (330, 1), (338, 0), (342, 0)

In [13]:
dough = Dough(length=500)
for _, row in df.iterrows():
    dough.add_defect(position=row['x'], defect_class=row['class'])
    
dough.place_biscuits_GA(best_solution)
print(dough)

Dough Defects:
Position: 355.45, Class: c
Position: 92.50, Class: a
Position: 141.88, Class: c
Position: 431.83, Class: c
Position: 435.03, Class: c
Position: 205.80, Class: a
Position: 34.69, Class: b
Position: 443.57, Class: a
Position: 69.42, Class: a
Position: 301.28, Class: a
Position: 342.65, Class: c
Position: 216.12, Class: a
Position: 367.67, Class: c
Position: 93.13, Class: a
Position: 401.42, Class: c
Position: 74.25, Class: c
Position: 356.84, Class: a
Position: 457.42, Class: a
Position: 395.64, Class: c
Position: 368.89, Class: a
Position: 163.92, Class: a
Position: 484.16, Class: b
Position: 55.27, Class: c
Position: 171.56, Class: b
Position: 119.69, Class: a
Position: 332.64, Class: b
Position: 85.10, Class: a
Position: 289.68, Class: b
Position: 156.79, Class: b
Position: 143.53, Class: c
Position: 83.61, Class: a
Position: 108.52, Class: a
Position: 57.58, Class: c
Position: 9.96, Class: c
Position: 440.00, Class: b
Position: 146.01, Class: b
Position: 259.33, Class:

In [14]:
biscuits_info = {
        0: Biscuit(0),
        1: Biscuit(1),
        2: Biscuit(2),
        3: Biscuit(3),
        4: Biscuit(4)
    }

In [15]:
dough.calculate_own_value(biscuits_info)

554

In [16]:
dough.validate_biscuits_placement(biscuits_info)

(True, 'All biscuits are correctly placed and within defect thresholds.')

The output value is preetty low, but that could be expected since our population is pretty small (50) and the number of defects far exceeds that, we would need a bigger population to reinforce our results or increase our mutation rates to find by hazard a better solution

# 2. Genetic Algorithm with Elitism Roulette

In [17]:
def main():
    dough = Dough(length=500) 
    for _, row in df.iterrows():
        dough.add_defect(position=row['x'], defect_class=row['class'])
    biscuits = {
        0: Biscuit(0),
        1: Biscuit(1),
        2: Biscuit(2),
        3: Biscuit(3)
    }
    
    GA = GeneticElitism(dough, biscuits, population_size=50, mutation_rate=0.01, crossover_rate=0.7)
    GA.initialize_population()

    for generation in range(100): 
        print(generation)
        GA.evolve()

    # Find the best solution in the final population
    best_solution = max(GA.population, key=GA.fitness)
    total_value = GA.fitness(best_solution)
    # [biscuit1, 0, 0, biscuit2]
    #dough.calculate_total_value(best_solution)
    print("Total Value:", total_value)
    print("Best Solution:", best_solution)
    return best_solution

In [18]:
if __name__ == "__main__":
    best_solution = main()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Total Value: 644
Best Solution: [(0, 1), (487, 1), (8, 1), (16, 1), (24, 3), (276, 1), (29, 3), (349, 3), (36, 2), (34, 2), (40, 1), (48, 2), (50, 0), (54, 2), (56, 1), (64, 0), (68, 1), (104, 0), (465, 3), (84, 3), (89, 1), (98, 0), (102, 2), (76, 1), (108, 3), (114, 0), (118, 0), (122, 1), (130, 1), (138, 2), (140, 2), (142, 0), (146, 1), (154, 2), (158, 1), (167, 3), (172, 2), (174, 1), (182, 2), (255, 3), (185, 2), (344, 3), (265, 3), (187, 1), (195, 3), (250, 3), (200, 1), (208, 3), (213, 3), (317, 3), (218, 3), (223, 1), (231, 3), (236, 2), (238, 2), (461, 0), (242, 1), (284, 3), (271, 3), (260, 3), (289, 2), (293, 1), (291, 2), (301, 1), (309, 1), (406, 1), (322, 0), (326, 1), (334, 3), (339, 3

# 3. Genetic Algorithm with Tournament Selection and Crossover

In [19]:
def main():
    biscuits = {
        0: Biscuit(0),
        1: Biscuit(1),
        2: Biscuit(2),
        3: Biscuit(3),
        4: Biscuit(4)
    }
    dough = Dough(500)

    for _, row in df.iterrows():
        dough.add_defect(position=row['x'], defect_class=row['class'])

    population_size = 150
    mutation_rate = 0.1
    crossover_rate = 0.5
    GA = GeneticTournament(dough, biscuits, population_size, mutation_rate, crossover_rate)
    num_generations = 100
    
    for generation in range(num_generations):
        print(generation)
        GA.evolve()

    best_solution = max(GA.population, key=GA.fitness)
    total_value = GA.fitness(best_solution)
    print("Best Solution Total Value:", total_value)
    print("Best Solution Configuration:", best_solution)
    return best_solution

if __name__ == "__main__":
    best_solution = main()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Best Solution Total Value: 633
Best Solution Configuration: [(106, 0), (281, 3), (321, 3), (417, 3), (54, 3), (122, 1), (8, 1), (24, 3), (213, 1), (111, 0), (29, 3), (393, 3), (34, 2), (36, 0), (383, 3), (473, 3), (353, 0), (410, 2), (286, 0), (293, 1), (48, 2), (478, 0), (195, 3), (50, 0), (93, 0), (130, 0), (434, 1), (59, 3), (365, 2), (402, 1), (72, 1), (80, 1), (250, 3), (88, 3), (450, 3), (313, 1), (0, 1), (16, 1), (455, 0), (98, 1), (345, 1), (469, 0), (118, 0), (388, 3), (290, 2), (464, 3), (134, 1), (495, 3), (381, 2), (146, 1), (40, 1), (373, 1), (208, 3), (158, 1), (430, 0), (255, 1), (273, 0), (175, 1), (268, 3), (185, 2), (200, 1), (167, 0), (142, 0), (412, 3), (171, 0), (333, 1), (154, 2)

In [20]:
biscuits = {
        0: Biscuit(0),
        1: Biscuit(1),
        2: Biscuit(2),
        3: Biscuit(3)
    }
dough = Dough(500)
for _, row in df.iterrows():
    dough.add_defect(position=row['x'], defect_class=row['class'])
dough.place_biscuits_GA(best_solution)

In [21]:
print(dough)

Dough Defects:
Position: 355.45, Class: c
Position: 92.50, Class: a
Position: 141.88, Class: c
Position: 431.83, Class: c
Position: 435.03, Class: c
Position: 205.80, Class: a
Position: 34.69, Class: b
Position: 443.57, Class: a
Position: 69.42, Class: a
Position: 301.28, Class: a
Position: 342.65, Class: c
Position: 216.12, Class: a
Position: 367.67, Class: c
Position: 93.13, Class: a
Position: 401.42, Class: c
Position: 74.25, Class: c
Position: 356.84, Class: a
Position: 457.42, Class: a
Position: 395.64, Class: c
Position: 368.89, Class: a
Position: 163.92, Class: a
Position: 484.16, Class: b
Position: 55.27, Class: c
Position: 171.56, Class: b
Position: 119.69, Class: a
Position: 332.64, Class: b
Position: 85.10, Class: a
Position: 289.68, Class: b
Position: 156.79, Class: b
Position: 143.53, Class: c
Position: 83.61, Class: a
Position: 108.52, Class: a
Position: 57.58, Class: c
Position: 9.96, Class: c
Position: 440.00, Class: b
Position: 146.01, Class: b
Position: 259.33, Class:

In [22]:
biscuits_info = {
        0: Biscuit(0),
        1: Biscuit(1),
        2: Biscuit(2),
        3: Biscuit(3),
        4: Biscuit(4)
    }
dough.calculate_own_value(biscuits_info)

633

In [23]:
dough.validate_biscuits_placement(biscuits_info)

(True, 'All biscuits are correctly placed and within defect thresholds.')

# 4. Uniform Crossover

In [24]:
def main():
    biscuits = {
        0: Biscuit(0),
        1: Biscuit(1),
        2: Biscuit(2),
        3: Biscuit(3),
        4: Biscuit(4)
    }
    dough = Dough(500)

    for _, row in df.iterrows():
        dough.add_defect(position=row['x'], defect_class=row['class'])

    population_size = 150
    mutation_rate = 0.1
    crossover_rate = 0.5
    GA = UniformCrossoverGA(dough, biscuits, population_size, mutation_rate, crossover_rate)
    num_generations = 100
    
    for generation in range(num_generations):
        print(generation)
        GA.evolve()

    best_solution = max(GA.population, key=GA.fitness)
    total_value = GA.fitness(best_solution)
    print("Best Solution Configuration:", best_solution)
    
    return best_solution

if __name__ == "__main__":
    best_solution = main()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Best Solution Configuration: [(0, 1), (8, 1), (16, 0), (20, 0), (24, 3), (29, 3), (34, 2), (36, 0), (40, 1), (48, 2), (50, 0), (54, 0), (58, 1), (66, 3), (71, 2), (73, 1), (81, 1), (89, 1), (98, 0), (102, 3), (107, 0), (111, 2), (114, 0), (118, 0), (122, 1), (130, 3), (135, 0), (139, 2), (141, 0), (145, 1), (153, 2), (155, 2), (158, 3), (164, 2), (167, 1), (175, 0), (179, 3), (185, 2), (187, 1), (195, 3), (200, 3), (205, 3), (210, 1), (218, 0), (222, 3), (227, 2), (229, 1), (237, 1), (245, 2), (247, 3), (252, 3), (257, 3), (262, 0), (266, 3), (271, 0), (275, 0), (279, 0), (283, 3), (288, 3), (293, 1), (301, 1), (309, 2), (311, 0), (315, 0), (319, 1), (327, 1), (335, 3), (340, 3), (345, 2), (347, 1), (

In [25]:
biscuits_info = {
        0: Biscuit(0),
        1: Biscuit(1),
        2: Biscuit(2),
        3: Biscuit(3),
        4: Biscuit(4)
    }

dough = Dough(500)
for _, row in df.iterrows():
    dough.add_defect(position=row['x'], defect_class=row['class'])
dough.place_biscuits_GA(best_solution)
print(dough)

Dough Defects:
Position: 355.45, Class: c
Position: 92.50, Class: a
Position: 141.88, Class: c
Position: 431.83, Class: c
Position: 435.03, Class: c
Position: 205.80, Class: a
Position: 34.69, Class: b
Position: 443.57, Class: a
Position: 69.42, Class: a
Position: 301.28, Class: a
Position: 342.65, Class: c
Position: 216.12, Class: a
Position: 367.67, Class: c
Position: 93.13, Class: a
Position: 401.42, Class: c
Position: 74.25, Class: c
Position: 356.84, Class: a
Position: 457.42, Class: a
Position: 395.64, Class: c
Position: 368.89, Class: a
Position: 163.92, Class: a
Position: 484.16, Class: b
Position: 55.27, Class: c
Position: 171.56, Class: b
Position: 119.69, Class: a
Position: 332.64, Class: b
Position: 85.10, Class: a
Position: 289.68, Class: b
Position: 156.79, Class: b
Position: 143.53, Class: c
Position: 83.61, Class: a
Position: 108.52, Class: a
Position: 57.58, Class: c
Position: 9.96, Class: c
Position: 440.00, Class: b
Position: 146.01, Class: b
Position: 259.33, Class:

In [26]:
dough.calculate_own_value(biscuits_info)

645

In [27]:
dough.validate_biscuits_placement(biscuits_info)

(True, 'All biscuits are correctly placed and within defect thresholds.')

# Greedy Search

In [28]:
def greedy_biscuit_placement(dough, biscuits, heuristic='value'):
    """
    Place biscuits on the dough using a greedy approach based on a specified heuristic.

    Parameters:
    - dough: The Dough object representing the dough roll.
    - biscuits: A dictionary of Biscuit objects indexed by their type.
    - heuristic: The heuristic to use for sorting biscuits. Options are:
        - 'value': Sort biscuits by their value in descending order.
        - 'valength': Sort biscuits by their value-to-length ratio in descending order.
        - Other: Sort biscuits by value divided by (length * sum of max defects), in descending order.

    Returns:
    - total_value: The total value of the biscuits placed, minus the penalty for unused dough.
    - placed_biscuits: A list of tuples representing the placed biscuits as (position, biscuit_type).
    """
    # Sort biscuits based on the selected heuristic
    if heuristic == 'value':
        sorted_biscuits = sorted(biscuits.values(), key=lambda b: b.value, reverse=True)
    elif heuristic == 'valength':
        sorted_biscuits = sorted(biscuits.values(), key=lambda b: b.value / b.length, reverse=True)
    else:
        sorted_biscuits = sorted(
            biscuits.values(),
            key=lambda b: b.value / (b.length * sum(b.max_defects.values())),
            reverse=True
        )

    total_value = 0  # Initialize total value
    current_position = 0  # Start at the beginning of the dough
    placed_biscuits = []  # List to store placed biscuits
    unused_positions = set(range(dough.LENGTH))  # Set of all positions on the dough

    while current_position < dough.LENGTH:
        placed = False  # Flag to check if a biscuit has been placed at the current position
        for biscuit in sorted_biscuits:
            # Check if the biscuit fits within the remaining dough length
            if current_position + biscuit.length <= dough.LENGTH:
                # Count defects in the current segment of the dough
                defects = dough.count_defects(current_position, biscuit.length)
                # Check if defects are within the biscuit's maximum allowed defects
                if all(defects.get(cls, 0) <= biscuit.max_defects[cls] for cls in biscuit.max_defects):
                    # Place the biscuit
                    placed_biscuits.append((current_position, biscuit.biscuit_type))
                    total_value += biscuit.value  # Add the biscuit's value
                    # Mark positions as used
                    for pos in range(current_position, current_position + biscuit.length):
                        unused_positions.discard(pos)
                    # Move current position forward by the biscuit's length
                    current_position += biscuit.length
                    placed = True
                    break  # Exit the loop after placing a biscuit
        if not placed:
            # No biscuit could be placed; move to the next position
            current_position += 1
            unused_positions.discard(current_position - 1)  # Mark this position as unused

    # Calculate the penalty for unused positions
    penalty = -len(unused_positions)
    total_value += penalty

    return total_value, placed_biscuits


In [29]:
def main(heuristic):
    dough = Dough(length=500)
    for _, row in df.iterrows():
        dough.add_defect(position=row['x'], defect_class=row['class'])
        
    biscuits = {
        0: Biscuit(0),
        1: Biscuit(1),
        2: Biscuit(2),
        3: Biscuit(3)
    }
    
    total_value, placed_biscuits = greedy_biscuit_placement(dough, biscuits,heuristic)
    print("Total Value:", total_value)
    print("Placed Biscuits:", placed_biscuits)

In [30]:
if __name__ == "__main__":
    main('value')

Total Value: 675
Placed Biscuits: [(0, 1), (8, 1), (16, 1), (24, 1), (32, 2), (34, 2), (36, 0), (40, 1), (48, 2), (50, 0), (54, 1), (62, 1), (70, 1), (78, 1), (86, 1), (95, 2), (98, 1), (106, 0), (111, 0), (118, 0), (122, 1), (130, 1), (138, 0), (142, 0), (146, 1), (154, 2), (158, 1), (167, 1), (175, 1), (185, 2), (187, 1), (195, 1), (203, 1), (211, 1), (219, 1), (227, 1), (235, 3), (242, 1), (250, 1), (258, 1), (266, 1), (274, 0), (278, 1), (286, 1), (294, 1), (302, 1), (310, 1), (318, 1), (326, 1), (334, 1), (342, 1), (351, 2), (353, 0), (357, 1), (365, 0), (369, 0), (373, 1), (381, 2), (383, 1), (391, 1), (399, 1), (407, 1), (415, 1), (423, 1), (431, 0), (435, 1), (443, 1), (451, 1), (459, 1), (467, 1), (475, 0), (479, 1), (487, 1), (495, 3)]


In [31]:
if __name__ == "__main__":
    main('valength')

Total Value: 690
Placed Biscuits: [(0, 1), (8, 1), (16, 3), (21, 1), (29, 3), (34, 2), (36, 0), (40, 1), (48, 2), (50, 0), (54, 3), (59, 3), (64, 1), (72, 1), (80, 1), (88, 3), (93, 0), (98, 1), (106, 0), (111, 0), (118, 0), (122, 1), (130, 3), (135, 0), (139, 0), (143, 0), (147, 3), (152, 3), (158, 3), (164, 2), (167, 3), (172, 3), (177, 0), (181, 2), (185, 2), (187, 1), (195, 3), (200, 3), (205, 3), (210, 3), (215, 3), (220, 3), (225, 3), (230, 3), (235, 3), (242, 1), (250, 3), (255, 3), (260, 3), (265, 3), (270, 3), (275, 0), (279, 1), (287, 3), (293, 1), (301, 1), (309, 3), (314, 1), (322, 3), (327, 3), (332, 2), (334, 3), (339, 3), (344, 3), (349, 3), (354, 1), (362, 0), (366, 0), (370, 0), (374, 1), (382, 1), (390, 3), (395, 3), (400, 1), (408, 3), (413, 3), (418, 3), (423, 1), (431, 0), (435, 1), (443, 1), (451, 3), (456, 3), (461, 3), (466, 3), (471, 3), (476, 0), (480, 1), (488, 3), (493, 3)]


In [32]:
if __name__ == "__main__":
    main('valengthdef')

Total Value: 672
Placed Biscuits: [(0, 1), (8, 1), (16, 3), (21, 2), (23, 1), (31, 3), (36, 2), (40, 1), (48, 2), (50, 0), (54, 3), (59, 3), (64, 1), (72, 2), (74, 1), (82, 1), (90, 2), (92, 0), (98, 1), (106, 2), (108, 3), (114, 2), (118, 0), (122, 2), (124, 1), (132, 3), (137, 2), (139, 2), (141, 0), (145, 1), (153, 2), (155, 2), (158, 3), (164, 2), (167, 3), (172, 3), (177, 0), (181, 2), (185, 2), (187, 1), (195, 3), (200, 3), (205, 3), (210, 3), (215, 3), (220, 3), (225, 3), (230, 3), (235, 3), (242, 1), (250, 3), (255, 3), (260, 3), (265, 3), (270, 3), (275, 0), (279, 1), (287, 3), (293, 1), (301, 1), (309, 3), (314, 1), (322, 3), (327, 3), (332, 2), (334, 3), (339, 3), (344, 3), (349, 3), (354, 1), (362, 2), (364, 0), (368, 0), (373, 1), (381, 2), (383, 3), (388, 3), (393, 3), (398, 2), (400, 1), (408, 3), (413, 3), (418, 3), (423, 2), (425, 1), (433, 1), (441, 3), (446, 1), (454, 3), (459, 3), (464, 3), (469, 3), (474, 2), (476, 2), (478, 0), (483, 1), (491, 3), (496, 0)]


# CSP

# 1. Import CSP librairies

In [33]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()

# 2. Create variables

In [34]:
# Define the Biscuit types
biscuit_types = [Biscuit(i) for i in range(4)]  # Assuming biscuit types are 0 to 4

# Print the details of each Biscuit type
for biscuit in biscuit_types:
    print(biscuit)

# Determine the maximum starting position for biscuits
# This ensures that even the largest biscuit can fit within the dough length
max_biscuit_length = max(biscuit.length for biscuit in biscuit_types)
max_start_pos = dough.LENGTH - max_biscuit_length

# Create variables for the CSP model
# biscuit_vars[(biscuit_type, start_pos)] = 1 if biscuit of type 'biscuit_type' starts at 'start_pos', 0 otherwise
biscuit_vars = {}
for biscuit in biscuit_types:
    for start_pos in range(max_start_pos + 1):
        var_name = f"biscuit_{biscuit.biscuit_type}_start_{start_pos}"
        biscuit_vars[(biscuit.biscuit_type, start_pos)] = model.NewBoolVar(var_name)


Biscuit Type: 0, Length: 4, Value: 3, Max Defects: {'a': 4, 'b': 2, 'c': 3}
Biscuit Type: 1, Length: 8, Value: 12, Max Defects: {'a': 5, 'b': 4, 'c': 4}
Biscuit Type: 2, Length: 2, Value: 1, Max Defects: {'a': 1, 'b': 2, 'c': 1}
Biscuit Type: 3, Length: 5, Value: 8, Max Defects: {'a': 2, 'b': 3, 'c': 2}


In [35]:
print(biscuit_vars[(0,3)])

biscuit_0_start_3


# 3. Create Constraints

In [36]:
# Constraint: No overlapping biscuits can be placed on the dough

for position in range(dough.LENGTH):
    # List to collect variables of biscuits covering the current position
    overlapping_biscuits = []
    
    for biscuit in biscuit_types:
        # Calculate possible start positions for this biscuit type that cover the current position
        start_positions = range(
            max(0, position - biscuit.length + 1),
            min(position + 1, max_start_pos + 1)
        )
        
        for start_pos in start_positions:
            # Check if the biscuit starting at 'start_pos' covers the 'position'
            if start_pos <= position < start_pos + biscuit.length:
                # Add the biscuit variable to the list
                overlapping_biscuits.append(biscuit_vars[(biscuit.biscuit_type, start_pos)])
    
    if overlapping_biscuits:
        # At most one biscuit can cover each position
        model.Add(sum(overlapping_biscuits) <= 1)


# Constraint: Defects within each biscuit must not exceed the maximum allowed for that biscuit type

for biscuit in biscuit_types:
    for start_pos in range(max_start_pos + 1):
        # For each defect type (e.g., 'a', 'b', 'c')
        for defect_type, max_allowed in biscuit.max_defects.items():
            # List to hold positions of defects of this type within the biscuit's range
            defect_positions = []
            
            # Iterate over all defects on the dough
            for defect_position, defect_class in dough.defects_list:
                # Check if the defect is of the current type and within the biscuit's range
                if defect_class == defect_type and start_pos <= defect_position < start_pos + biscuit.length:
                    defect_positions.append(defect_position)
            
            # Number of defects of this type within the biscuit's range
            num_defects = len(defect_positions)
            
            # Add the constraint: If the biscuit is placed, the number of defects must not exceed the maximum allowed
            model.Add(num_defects * biscuit_vars[(biscuit.biscuit_type, start_pos)] <= max_allowed)


# 4. Objective function to maximize

In [37]:
# Create variables indicating whether each position on the dough is covered by a biscuit
position_covered = {}
for x in range(dough.LENGTH):
    position_covered[x] = model.NewBoolVar(f'position_covered_{x}')

# Link the position_covered variables with the biscuit placement variables
for x in range(dough.LENGTH):
    # List to collect variables of biscuits covering the current position
    covering_biscuits = []
    for biscuit in biscuit_types:
        # Calculate possible start positions for this biscuit type that cover the current position
        start_positions = range(
            max(0, x - biscuit.length + 1),
            min(x + 1, max_start_pos + 1)
        )
        for start_pos in start_positions:
            # Check if the biscuit starting at 'start_pos' covers the 'position'
            if start_pos <= x < start_pos + biscuit.length:
                covering_biscuits.append(biscuit_vars[(biscuit.biscuit_type, start_pos)])
    # Since at most one biscuit can cover each position, the sum of covering_biscuits is 0 or 1
    # Set position_covered[x] to be equal to this sum
    model.Add(position_covered[x] == sum(covering_biscuits))

# Calculate the total value of placed biscuits
total_biscuit_value = sum(
    biscuit.value * biscuit_vars[(biscuit.biscuit_type, start_pos)]
    for biscuit in biscuit_types
    for start_pos in range(max_start_pos + 1)
)

# Calculate the total number of positions covered by biscuits
total_positions_covered = sum(position_covered[x] for x in range(dough.LENGTH))

# Calculate the total penalty for unused positions
# Each unused position incurs a penalty of -1
# Total penalty = (dough.LENGTH - total_positions_covered)
total_penalty = dough.LENGTH - total_positions_covered

# Define the objective function to maximize
# Total value = total_biscuit_value - total_penalty
# Simplify the expression for better solver performance
total_value = total_biscuit_value - total_penalty

# Set the objective in the model
model.Maximize(total_value)


# Solve the problem

In [38]:
start =time.time()
solver = cp_model.CpSolver()
status = solver.Solve(model)
print("solved in : ", time.time()-start,"seconds")

solved in :  0.21929049491882324 seconds


In [39]:
if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    if status==cp_model.OPTIMAL:
        print("The solution found is optimal !")
    if status==cp_model.FEASIBLE:
        print("The solution found is feasible !")
solution = []
for biscuit in biscuit_types:
    for start_pos in range(max_start_pos + 1):
        if solver.Value(biscuit_vars[(biscuit.biscuit_type, start_pos)]) == 1:#if 1/True this mean that it has been placed
            solution.append((biscuit.biscuit_type, start_pos))
print("The objective function value found is : ",solver.ObjectiveValue()," and the solution found is : ",solution)

The solution found is optimal !
The objective function value found is :  715.0  and the solution found is :  [(0, 36), (0, 50), (0, 114), (0, 140), (0, 353), (0, 365), (0, 371), (1, 0), (1, 8), (1, 23), (1, 40), (1, 64), (1, 73), (1, 81), (1, 89), (1, 98), (1, 119), (1, 132), (1, 149), (1, 158), (1, 167), (1, 176), (1, 187), (1, 242), (1, 270), (1, 278), (1, 286), (1, 294), (1, 312), (1, 325), (1, 357), (1, 375), (1, 398), (1, 406), (1, 424), (1, 432), (1, 440), (1, 448), (1, 471), (1, 479), (1, 492), (2, 21), (2, 48), (2, 106), (2, 185), (2, 225), (2, 369), (3, 16), (3, 31), (3, 54), (3, 59), (3, 108), (3, 127), (3, 144), (3, 195), (3, 200), (3, 205), (3, 210), (3, 215), (3, 220), (3, 227), (3, 232), (3, 237), (3, 250), (3, 255), (3, 260), (3, 265), (3, 302), (3, 307), (3, 320), (3, 333), (3, 338), (3, 343), (3, 348), (3, 383), (3, 388), (3, 393), (3, 414), (3, 419), (3, 456), (3, 461), (3, 466), (3, 487)]


In [40]:
sorted_list = sorted(solution, key=lambda x: x[1])
print(sorted_list)

[(1, 0), (1, 8), (3, 16), (2, 21), (1, 23), (3, 31), (0, 36), (1, 40), (2, 48), (0, 50), (3, 54), (3, 59), (1, 64), (1, 73), (1, 81), (1, 89), (1, 98), (2, 106), (3, 108), (0, 114), (1, 119), (3, 127), (1, 132), (0, 140), (3, 144), (1, 149), (1, 158), (1, 167), (1, 176), (2, 185), (1, 187), (3, 195), (3, 200), (3, 205), (3, 210), (3, 215), (3, 220), (2, 225), (3, 227), (3, 232), (3, 237), (1, 242), (3, 250), (3, 255), (3, 260), (3, 265), (1, 270), (1, 278), (1, 286), (1, 294), (3, 302), (3, 307), (1, 312), (3, 320), (1, 325), (3, 333), (3, 338), (3, 343), (3, 348), (0, 353), (1, 357), (0, 365), (2, 369), (0, 371), (1, 375), (3, 383), (3, 388), (3, 393), (1, 398), (1, 406), (3, 414), (3, 419), (1, 424), (1, 432), (1, 440), (1, 448), (3, 456), (3, 461), (3, 466), (1, 471), (1, 479), (3, 487), (1, 492)]
